# Population based metaheuristics - Flowshop problem

This notebook focuses on implementing some population-based algorithms to solve the flowshop problem. Population-based algorithms may find high-quality solutions within a reasonable amount of time.

### Table of content
- [Required functions](#Required-functions)
- [Genetic Algorithm](#Genetic-Algorithm)
- [Ant Colony Algorithm](#Ant-Colony-Algorithm)
- [Greedy NEH](#Greedy-NEH)
- [Hybrid GA with VNS](#Hybrid-GA-with-VNS)
- [Tests](#Tests)

### References
- [Benchmarks for Basic Scheduling Problems](http://mistic.heig-vd.ch/taillard/articles.dir/Taillard1993EJOR.pdf)

# Required functions

In [1]:
import numpy as np
import matplotlib as plt
import itertools
import time
import pandas as pd
import math
import random

In [2]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)
    completion_times = np.zeros((num_jobs, num_machines))
    
    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]
    
    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]
    
    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

## CDS

In [3]:
def CDS_heuristic(processing_times):
    jobs, machines = processing_times.shape
    m = machines-1
    johnson_proc_times = np.zeros((jobs,2))
    best_cost = np.inf
    best_seq = []
    for k in range(m):
        johnson_proc_times[:,0] += processing_times[:,k]
        johnson_proc_times[:,1] += processing_times[:,-k-1]
        seq = johnson_method(johnson_proc_times)
        cost = evaluate_sequence(seq,processing_times)
        if cost < best_cost:
            best_cost = cost
            best_seq = seq
    return best_seq, best_cost

## Palmer

In [4]:
def palmer_heuristic(processing_times):
    jobs, machines = processing_times.shape
    f = []
    for i in range(jobs):
        fi = 0
        for j in range(machines):
            fi += (machines -2*j + 1) * processing_times[i][j]
        f.append(fi)
    order = sorted(range(jobs), key=lambda k: f[k])
    return order

## Gupta

In [5]:
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0
    
def min_gupta(job, processing_times):
    m = np.inf
    _, machines = processing_times.shape
    for i in range(machines-1):
        k = processing_times[job][i] + processing_times[job][i+1]
        if (k < m):
            m = k
    return k

def gupta_heuristic(processing_times):
    jobs, machines = processing_times.shape
    f = []
    for i in range(jobs):
        fi = sign(processing_times[i][0] - processing_times[i][machines-1]) / min_gupta(i,processing_times)
        f.append(fi)
    order = sorted(range(jobs), key=lambda k: f[k])
    return order

## Johnson

In [6]:
def johnson_method(processing_times):
    jobs, machines = processing_times.shape
    copy_processing_times = processing_times.copy()
    maximum = processing_times.max() + 1
    m1 = []
    m2 = []
    
    if machines != 2:
        raise Exception("Johson method only works with two machines")
        
    for i in range(jobs):
        minimum = copy_processing_times.min()
        position = np.where(copy_processing_times == minimum)
        
        if position[1][0] == 0:
            m1.append(position[0][0])
        else:
            m2.insert(0, position[0][0])
        
        copy_processing_times[position[0][0]] = maximum
        # Delete the job appended
        
    return m1+m2

## Artificial Heuristic

In [7]:
def artificial_heuristic(processing_times):
    jobs, machines = processing_times.shape
    r = 1
    best_cost = np.inf
    best_seq = []
    while r != machines :
        wi = np.zeros((jobs, machines - r))
        for i in range(jobs):
            for j in range(0, machines - r):
                wi[i, j] = (machines - r) - (j)
       
        am = np.zeros((jobs, 2))    
        am[:, 0] = np.sum(wi[:, :machines - r] * processing_times[:, :machines - r], axis=1)
        for i in range(jobs):
            for j in range(0, machines - r):
                am[i, 1] += wi[i, j ] * processing_times[i, machines - j - 1]

        seq = johnson_method(am)
        cost = evaluate_sequence(seq, processing_times)
        if cost < best_cost:
            best_cost = cost
            best_seq = seq
        r += 1
       
    return best_seq, best_cost

## NEH

In [8]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if evaluate_sequence([J1, J2], processing_times) < evaluate_sequence([J2, J1], processing_times) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = evaluate_sequence(new_sequence, processing_times)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

## VNS

In [9]:
def swap(solution, i, k):
    temp = solution[k]
    solution[k] = solution[i]
    solution[i] = temp
    return solution

In [10]:
def random_swap(solution, processing_times):
    i = np.random.choice(list(solution))
    k = np.random.choice(list(solution))
    # Generating two different random positions
    while (i == k):
        k = np.random.choice(list(solution))
    # Switch between job i and job k in the given sequence
    neighbor = solution.copy()
    return swap(neighbor, i, k), evaluate_sequence(neighbor, processing_times)

In [11]:
def best_first_swap(solution, processing_times):
    # This function will take a solution, and return the first best solution.
    # The first solution that is better then the current one 'solution' in args.
    num_jobs = len(solution)
    best_cmax = evaluate_sequence(solution, processing_times)
    best_neighbor = solution.copy()
    for k1 in range(num_jobs):
        for k2 in range(k1+1, num_jobs):
            neighbor = solution.copy()
            neighbor = swap(neighbor,k1,k2)
            cmax = evaluate_sequence(neighbor, processing_times)
            if cmax < best_cmax:
                best_neighbor = neighbor
                best_cmax = cmax
                return best_neighbor, best_cmax
    return best_neighbor, best_cmax

In [12]:
def best_swap(solution, processing_times):
    # This function will take a solution, and return its best neighbor solution.
    num_jobs = len(solution)
    best_cmax = np.Infinity
    for k1 in range(num_jobs):
        for k2 in range(k1+1, num_jobs):
            neighbor = solution.copy()
            neighbor = swap(neighbor,k1,k2)
            cmax = evaluate_sequence(neighbor, processing_times)
            if cmax < best_cmax:
                best_neighbor = neighbor
                best_cmax = cmax
    return best_neighbor, best_cmax

In [13]:
def best_swaps(solution, processing_times):
    # This function will take a solution, and return a list that contains all solutions that are better than it.
    num_jobs = len(solution)
    cmax = evaluate_sequence(solution, processing_times)
    bests = []
    for k1 in range(num_jobs):
        for k2 in range(k1+1, num_jobs):
            neighbor = solution.copy()
            swap(neighbor,k1,k2)
            neighbor_cmax = evaluate_sequence(neighbor, processing_times)
            if neighbor_cmax < cmax:
                bests.append((neighbor_cmax, neighbor))
    bests.sort(key=lambda x: x[0])
    return bests

In [14]:
def random_insertion(solution, processing_times):
    # This function consists of choosing random two indices, i and k.
    # Remove the element at indice i, and insert it in the position k.
    i = np.random.choice(list(solution))
    k = np.random.choice(list(solution))
    while (i == k):
        k = np.random.choice(list(solution))
    neighbor = solution.copy()
    neighbor.remove(solution[i])
    neighbor.insert(k, solution[i])
    return neighbor, evaluate_sequence(neighbor, processing_times)

In [15]:
def best_insertion(solution, processing_times):
    # This function consists of trying all different insertions.
    # Then it returns the best one among them
    num_jobs = len(solution)
    best_cmax = np.Infinity
    for k1 in range(num_jobs):
        s = solution.copy()
        s_job = solution[k1]
        s.remove(s_job)
        for k2 in range(num_jobs):
            if k1 != k2:
                neighbor = s.copy()
                neighbor.insert(k2, s_job)
                cmax = evaluate_sequence(neighbor, processing_times)
                if cmax < best_cmax:
                    best_neighbor = neighbor
                    best_cmax = cmax
    return best_neighbor, best_cmax

In [16]:
def best_edge_insertion(solution, processing_times):
    num_jobs = len(solution)
    best_cmax = np.Infinity
    for k1 in range(num_jobs-1):
        s = solution.copy()
        s_job1 = s[k1] 
        s_job2 = s[k1+1]
        s.remove(s_job1)
        s.remove(s_job2)
        for k2 in range(num_jobs-1):
            if(k1 != k2):
                neighbor = s.copy()
                neighbor.insert(k2, s_job1)
                neighbor.insert(k2+1, s_job2)
                cmax = evaluate_sequence(neighbor, processing_times)
                if cmax < best_cmax:
                    best_neighbor = neighbor
                    best_cmax = cmax
    return best_neighbor, best_cmax

In [17]:
def get_neighbor(solution, processing_times, method="random_swap"):
    # Swapping methods
    if method == "random_swap":
        neighbor, cost = random_swap(solution, processing_times)
    elif method == "best_swap":
        neighbor, cost = best_swap(solution, processing_times)
    elif method == "best_first_swap":
        neighbor, cost = best_first_swap(solution, processing_times)
    # Insertion methods
    elif method == "random_insertion":
        neighbor, cost = random_insertion(solution, processing_times)
    elif method == "best_edge_insertion":
        neighbor, cost = best_edge_insertion(solution, processing_times)
    elif method == "best_insertion":
        neighbor, cost = best_insertion(solution, processing_times)
    # Randomly pick a method of generating neighbors.
    else:     
        i = random.randint(0, 5)
        if i == 0:
            neighbor, cost = random_swap(solution, processing_times)
        elif i == 1:
            neighbor, cost = best_swap(solution, processing_times)
        elif i == 2:
            neighbor, cost = best_first_swap(solution, processing_times)
        elif i == 3:
            neighbor, cost = random_insertion(solution, processing_times)
        elif i == 4:
            neighbor, cost = best_edge_insertion(solution, processing_times)
        else:
            neighbor, cost = best_insertion(solution, processing_times)
    return neighbor, cost

In [18]:
# This function performs a random perturbation on a list by swapping the elements of k adjacent positions.
# solution: the list to be perturbed
# k: the number of adjacent positions to be swapped
def shake_VNS(solution, k):
    n = len(solution)
    # If k is greater than the length of the list, then we create perturbations on all elements.
    indices = random.sample(range(n), min(k, n-1))
    indices.sort()
    neighbor = solution.copy()
    for i in indices:
        j = (i+k) % n
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

In [19]:
def vns(sol_init, processing_times, max_iterations, k_max):
    num_jobs = len(sol_init)
    current_solution = sol_init
    current_cost = evaluate_sequence(current_solution, processing_times)
    k = 1
    iteration = 0
    while iteration < max_iterations:       
        # Appliquer la perturbation "shake" pour générer le voisin aléatoire
        best_neighbor_solution = shake_VNS(current_solution, k)
        best_neighbor_cost = evaluate_sequence(best_neighbor_solution, processing_times)
        
        # Parcourir les voisins pour trouver la meilleure solution locale
        for l in range(1, k_max+1):
            neighbor, neighbor_cost  = get_neighbor(current_solution, processing_times)
            if (neighbor_cost < best_neighbor_cost):
                best_neighbor_solution = neighbor
                best_neighbor_cost = neighbor_cost            
        
        # Si la meilleure solution trouvée dans le voisinage est meilleure que la solution courante
        # alors on met à jour la solution courante et on réinitialise le rayon de la recherche
        if ( best_neighbor_cost < current_cost ):
            current_solution = best_neighbor_solution
            current_cost = best_neighbor_cost
            k = 1
        else:
            k += 1
        iteration += 1
    return current_solution, current_cost

# Genetic Algorithm

In [20]:
def selection_GA(population, processing_times, n_selected, strategie):
    # case "roulette":
    if strategie == "roulette":
        fitness = [evaluate_sequence(seq, processing_times) for seq in population]
        fitness_sum = sum(fitness)
        selection_probs = [fitness[i]/fitness_sum for i in range(len(population))]
        cum_probs = [sum(selection_probs[:i+1]) for i in range(len(population))]
        selected = []
        for i in range(n_selected):
            while True:
                rand = random.random()
                for j, cum_prob in enumerate(cum_probs):
                    if rand < cum_prob:
                        break
                if population[j] not in selected:
                    selected.append(population[j])
                    break
    # case "Elitism":
    elif strategie == "Elitism":
        fitness = [evaluate_sequence(seq, processing_times) for seq in population]
        sorted_population = [x for x, _ in sorted(zip(population, fitness), key=lambda pair: pair[1], reverse=False)]
        selected = sorted_population[:n_selected]

    # case "rank":
    elif strategie == "rank":
        fitness = [evaluate_sequence(seq, processing_times) for seq in population]
        sorted_population = sorted(population, key = lambda x: fitness[population.index(x)])
        fitness_sum = sum(i+1 for i in range(len(sorted_population)))
        selection_probs = [(len(sorted_population)-i)/fitness_sum for i in range(len(sorted_population))]
        selected = []
        for i in range(n_selected):
            selected_index = random.choices(range(len(sorted_population)), weights=selection_probs)[0]
            selected.append(sorted_population[selected_index])
            sorted_population.pop(selected_index)
            selection_probs.pop(selected_index)
            
    # case "tournament":
    elif strategie == "tournament":
        k = 2
        selected = []
        for i in range(n_selected):
            while True:
                tournament = random.sample(population, k)
                tournament = [seq for seq in tournament if seq not in selected]
                if tournament:
                    break
            fitness = [evaluate_sequence(seq, processing_times) for seq in tournament]
            selected.append(tournament[fitness.index(min(fitness))])

    return selected

In [21]:
def crossover_GA(p1, p2, points):
    jobs = len(p1) - 1
    # One points crossover
    if points == 'ONE':
        point = random.randint(0, jobs)
        offspring1 = p1[:point] + p2[point:]
        offspring2 = p2[:point] + p1[point:]
        points = [point]
    else: # Two Points crossover
        point_1 = random.randint(0, jobs)
        point_2 = random.randint(0, jobs)
        if point_1 > point_2:
            point_1, point_2 = point_2, point_1
        offspring1 = p1[:point_1] + p2[point_1:point_2] + p1[point_2:]
        offspring2 = p2[:point_1] + p1[point_1:point_2] + p2[point_2:]
        points = [point_1, point_2]
    # Remove duplicates and replace with genes from the other offspring
    offspring1 = remove_duplicates_GA(offspring1, offspring2, points)
    offspring2 = remove_duplicates_GA(offspring2, offspring1, points)
    return offspring1, offspring2

def remove_duplicates_GA(offspring, other_offspring, points):
    jobs = len(offspring) - 1
    check_points = len(points) > 1
    while True:
        duplicates = set([job for job in offspring if offspring.count(job) > 1])
        if not duplicates:
            break
        for job in duplicates:
            pos = [i for i, x in enumerate(offspring) if x == job]
            if (check_points and ((pos[0] < points[0]) or (pos[0] >= points[1])) ) or  ( (pos[0] < points[0]) and not check_points):
                dup = pos[0]
                index = pos[1]
            else:
                dup = pos[1]
                index = pos[0]

            offspring[dup] = other_offspring[index]
    return offspring

In [22]:
def mutation_GA(sequence, mutation_rate):
    num_jobs = len(sequence)
    for i in range(num_jobs):
        r = random.random()
        if r < mutation_rate:
            available_jobs = [j for j in range(num_jobs) if j != sequence[i]]
            newjob = random.sample(available_jobs, 1)[0]
            sequence[sequence.index(newjob)] = sequence[i]
            sequence[i] = newjob
    return sequence

In [23]:
def genetic_algorithm(processing_times, init_pop, pop_size, select_pop_size, selection_method, cossover, mutation_probability, num_iterations):
    # Init population generation
    population = init_pop
    best_seq = selection_GA(population, processing_times, 1, "Elitism")[0]
    best_cost = evaluate_sequence(best_seq, processing_times)
    for i in range(num_iterations):
        # Selection
        s = int(select_pop_size * pop_size) # number of selected individus to be parents (%)
        parents = selection_GA(population, processing_times, s, selection_method)
        # Crossover
        new_generation = []
        for _ in range(0, pop_size, 2):
            parent1 = random.choice(parents)
            parent2 = random.choice([p for p in parents if p != parent1])
            child1, child2 = crossover_GA(parent1, parent2, cossover)
            new_generation.append(child1)
            new_generation.append(child2)

        new_generation = new_generation[:pop_size]
        # Mutation
        for i in range(pop_size):
            if random.uniform(0, 1) < mutation_probability:
                new_generation[i] = mutation_GA(new_generation[i], mutation_probability)
        # Replacement
        population = new_generation

        # checking for best seq in current population
        best_seq_pop = selection_GA(population, processing_times, 1, "Elitism")[0]
        best_cost_pop = evaluate_sequence(best_seq_pop, processing_times)
        if best_cost_pop < best_cost:
            best_seq = best_seq_pop.copy()
            best_cost = best_cost_pop

    return best_seq, best_cost   

# Ant Colony Algorithm

In [24]:
def distance(i, j, processing_times):
    m = processing_times.shape[1]
    max_delay = 0
    for k in range(2, m):
        delay = np.sum(processing_times[i,1:k]) - np.sum(processing_times[j,1:k-1])
        if delay > max_delay:
            max_delay = delay
    return (processing_times[i, 0] + max(0, max_delay))

def ant_colony_optimization(num_ants, num_iterations, alpha, beta, evaporation_rate, Q, tau0, q0, processing_times):
    num_jobs = processing_times.shape[0]
    tau = np.ones((num_jobs, num_jobs)) * tau0
    best_schedule = None
    best_makespan = np.inf
    
    # visibility
    heuristic_values = np.array([ [1/distance(i, j, processing_times) if j != i else 0 for j in range(num_jobs)] for i in range(num_jobs)])
    for iteration in range(num_iterations): 
        sequences = []
        for ant in range(num_ants):
            current_job = np.random.randint(num_jobs)
            current_sequence = [current_job] 
            for job in range(num_jobs-1):
                unscheduled = [j for j in range(num_jobs) if j not in current_sequence]
                probabilities = np.zeros(len(unscheduled))
                
                for i, unscheduled_job in enumerate(unscheduled):
                    probabilities[i] = (tau[current_job, unscheduled_job]**alpha) * (heuristic_values[current_job, unscheduled_job]**beta)
                
                probabilities /= np.sum(probabilities)
                
                # pseudo-random-proportional
                q = random.random()
                if q < q0: # exploitation : choose the best
                    next_job = unscheduled[np.where( probabilities == np.max(probabilities))[0][0]]
                else: # exploration
                    next_job = np.random.choice(unscheduled, p=probabilities)
                    
                current_sequence.append(next_job)
                current_point = next_job
            
            makespan = evaluate_sequence(current_sequence, processing_times)
            sequences.append((current_sequence, makespan))
            
            if makespan < best_makespan:
                best_makespan = makespan
                best_schedule = current_sequence.copy()
        
        pheromone_delta =  np.zeros((num_jobs, num_jobs))
        for ant in range(num_ants):
            seq, ev = sequences[ant]
            for j in range(num_jobs-1):
                pheromone_delta[seq[j], seq[j+1]] += Q / ev
                             
        # Update pheromone  
        tau = tau * evaporation_rate + pheromone_delta
    
    return best_schedule, best_makespan

# Greedy NEH

At each step of the NEH method, instead of getting the best partial solution as the original NEH dictates, one out of the best five partial solutions is randomly chosen with equal probabilities. This way, high-quality individuals, yet different from each other, can be generated.

In [25]:
def greedy_neh_algorithm(processing_times, num_candidates, num_iterations):
    n = len(processing_times)
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    best_sequence = []
    best_cmax = float('inf')
    for i in range(num_iterations):
        partial_sequence = [ordered_sequence[0]]
        for k in range(1, n):
            candidates = []
            for job in ordered_sequence:
                if job not in partial_sequence:
                    for i in range(k+1):
                        candidate_sequence = insertion(partial_sequence, i, job)
                        candidate_cmax = evaluate_sequence(candidate_sequence, processing_times)
                        candidates.append((candidate_sequence, candidate_cmax))
            candidates.sort(key=lambda x: x[1])
            partial_sequence, cmax = random.choice(candidates[:num_candidates])
        if cmax < best_cmax:
                best_sequence = partial_sequence
                best_cmax = cmax
        ordered_sequence.append(ordered_sequence.pop(0))
    return best_sequence, best_cmax

We implement a second GRNEH that does not include the notion of iteration, which ultimately provides a solution that approaches optimality, in order to use it in population generation later.

In [26]:
def GRNEH(processing_times, num_candidates):
    n = len(processing_times)
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    partial_sequence = [ordered_sequence[0]]
    for k in range(1, n):
        candidates = []
        for job in ordered_sequence:
            if job not in partial_sequence:
                for i in range(k+1):
                    candidate_sequence = insertion(partial_sequence, i, job)
                    candidate_cmax = evaluate_sequence(candidate_sequence, processing_times)
                    candidates.append((candidate_sequence, candidate_cmax))
        candidates.sort(key=lambda x: x[1])
        partial_sequence, cmax = random.choice(candidates[:num_candidates])
    return partial_sequence, cmax

# Hybrid GA with VNS

## Generating an initial population

For our genetic algorithm, we implement a function to generate the initial population.
- **5** sequences generated by our heuristics
- **(a% * population size) - 5** sequences generated by GRNEH
- **(100 - a%) * population size** sequences generated randomly.

The article *Minimizing makespan in permutation flow shop scheduling problems using a hybrid metaheuristic algorithm* by **Zobolas et al** explains that the higher the value of a, the better the makespan of the final solution improves, but the longer the algorithm takes to converge.

In [62]:
def init_pop(processing_times, pop_size, a):
    
    #Les individus stockeront la séquence ( solution ), le makespan et un compteur c qui va déterminé si une solution est agée ou non
    population = []
    #Heuristics
    sol, cmax = neh_algorithm(processing_times)
    population.append(( sol, cmax ,0))
    sol, cmax = CDS_heuristic(processing_times)
    population.append(( sol, cmax ,0))
    sol=palmer_heuristic(processing_times)
    population.append((sol, evaluate_sequence(sol, processing_times),0))
    sol, cmax = artificial_heuristic(processing_times)
    population.append(( sol, cmax ,0))
    sol = gupta_heuristic(processing_times)
    population.append((sol, evaluate_sequence(sol, processing_times),0))
    
    #GRNEH
    for i in range(int(a*pop_size) - 5 ):
        sol, cmax = GRNEH(processing_times, 5)
        population.append(( sol, cmax ,0))
    
    #Random
    for i in range (int((1-a)*pop_size)):
        sol = np.random.permutation(processing_times.shape[0]).tolist() 
        population.append((sol, evaluate_sequence(sol, processing_times),0))
    
    return population

## Population improvement via GA

### Parents selection
To test our algorithm, we implemented four parent selection strategies, but we will use Tournament below because our algorithm includes an intensification phase, and the selection operator must ensure diversification

In [28]:
def selection(population, processing_times, n_selected, strategie):
    # case "roulette":
    if strategie == "roulette":
        fitness = [evaluate_sequence(seq, processing_times) for seq in population]
        fitness_sum = sum(fitness)
        selection_probs = [fitness[i]/fitness_sum for i in range(len(population))]
        cum_probs = [sum(selection_probs[:i+1]) for i in range(len(population))]
        selected = []
        for i in range(n_selected):
            while True:
                rand = random.random()
                for j, cum_prob in enumerate(cum_probs):
                    if rand < cum_prob:
                        break
                if population[j] not in selected:
                    selected.append(population[j])
                    break
    # case "Elitism":
    elif strategie == "Elitism":
        fitness = [evaluate_sequence(seq, processing_times) for seq in population]
        sorted_population = [x for x, _ in sorted(zip(population, fitness), key=lambda pair: pair[1], reverse=False)]
        selected = sorted_population[:n_selected]

    # case "rank":
    elif strategie == "rank":
        fitness = [evaluate_sequence(seq, processing_times) for seq in population]
        sorted_population = sorted(population, key = lambda x: fitness[population.index(x)])
        fitness_sum = sum(i+1 for i in range(len(sorted_population)))
        selection_probs = [(len(sorted_population)-i)/fitness_sum for i in range(len(sorted_population))]
        selected = []
        for i in range(n_selected):
            selected_index = random.choices(range(len(sorted_population)), weights=selection_probs)[0]
            selected.append(sorted_population[selected_index])
            sorted_population.pop(selected_index)
            selection_probs.pop(selected_index)
            
    # case "tournament":
    elif strategie == "tournament":
        #a random number of solutions is selected at first, ranging from 3 to 10.
        k = random.randint(3, 10)
        selected = []
        for i in range(n_selected):
            while True:
                tournament = random.sample(population, k)
                tournament = [seq for seq in tournament if seq not in selected]
                if tournament:
                    break
            selected.append(min(tournament, key=lambda x: x[1]))

    return selected

### Crossing of parents
In the article *Genetic algorithms for flowshop scheduling problems. Computers and Industrial Engineering* by **Murata T, Ishibuchi H, Tanaka H**, it was found that the best crossover operator for PFSP (Permutation Flowshop Scheduling Problems) is the **two-point crossover version I**.

In [29]:
def tp_crossover(parent1, parent2, processing_times):
    jobs = len(processing_times) - 1
    point1 = random.randint(0, jobs)
    point2 = random.randint(0, jobs)
    
    while point1 == point2 or point1 == (point2-1) or (point1 == point2+1) : 
        point2 = random.randint(0, jobs)
        
    if point1 > point2:
        point1, point2 = point2, point1

    # Create offspring as copies of parents
    offspring1 = parent1[0].copy()
    offspring2 = parent2[0].copy()

     # Set the segment between point1 and point2 to empty in offspring1
    offspring1[point1:point2] = [None] * (point2 - point1)

     # Set the segment between point1 and point2 to empty in offspring2
    offspring2[point1:point2] = [None] * (point2 - point1)
    
    # Remove duplicates from offspring1 and copy remaining genes from parent2
    for gene in parent2[0]:
         if gene not in offspring1:
            offspring1[offspring1.index(None)] = gene
            
    # Remove duplicates from offspring2 and copy remaining genes from parent1
    for gene in parent1[0]:
         if gene not in offspring2:
            offspring2[offspring2.index(None)] = gene
    
    return offspring1, offspring2

### Mutation of offspring
Both articles, **Ruiz R, Maroto C, Alcaraz J.**: *Two new robust genetic algorithms for the flowshop scheduling problem. Omega—International Journal of Management Science* and *Computers and Industrial Engineering* by **Murata T, Ishibuchi H, Tanaka H.**, have shown that the best operator for PFSP is a shift mutation, which means a random insertion.

In [30]:
def random_insertion(solution, processing_times):
    # This function consists of choosing random two indices, i and k.
    # Remove the element at indice i, and insert it in the position k.
    i = np.random.choice(list(solution))
    k = np.random.choice(list(solution))
    while (i == k):
        k = np.random.choice(list(solution))
    neighbor = solution.copy()
    neighbor.remove(solution[i])
    neighbor.insert(k, solution[i])
    return neighbor, evaluate_sequence(neighbor, processing_times)

### Updating the population
The best performing offspring are included in the new population because, according to **Ruiz R, Maroto C, and Alcaraz J.**, this selection performs better than directly replacing parents. Therefore, we replace the worst individuals in the population with the best unique offspring. In order to avoid premature convergence, we only accept a new solution in a population if it is not already present in the population.

In [31]:
def sol_unique(population, solution): 
    for p in population:
        if p[0] == solution:
            return False
    return True

def replace_sol(population, old_sol, new_sol):
    for i in range(len(population)):
        if (population[i] == old_sol):
            population[i] = new_sol
            return
    return  

def replace_with_offspring(population, offspring):
    #Vérifier que l'offspring n'existe pas déjà dans la population
    if (sol_unique(population, offspring[0])==False): 
        return

    #Si il est unique, on prend le pire individu de la population courante
    worst_ind = max(population, key=lambda x: x[1])
    if worst_ind[1] > offspring[1]:
        replace_sol(population, worst_ind, offspring)
        
    return  

## Complete genetic algorithm

In [32]:
def AG(population, processing_times):
    children = []
    #génération des enfants
    for i in range(int(len(population)/2)):
        mating_pool = selection(population, processing_times, 2, "tournament")
        os1, os2 = tp_crossover(mating_pool[0], mating_pool[1], processing_times)
        os1, fit1 = random_insertion(os1, processing_times)
        os2, fit2 = random_insertion(os2, processing_times)
        children.append((os1, fit1, 0))
        children.append((os2, fit2, 0))
   
    #remplacement dans la population
    for child in children: 
        replace_with_offspring(population, child)
    return

## Intensification
At this stage, we apply a VNS local search on a selected sub-population to complement the genetic algorithm

### VNS operators 
The main idea behind intensification here is that simpler neighborhood structures should be explored first (i.e., random swap and random insertion) before moving on to more complex ones (i.e., 2-opt and or-opt).

In [33]:
def swap(solution, i, k):
    temp = solution[k]
    solution[k] = solution[i]
    solution[i] = temp
    return solution

In [34]:
def random_swap(solution, processing_times):
    i = np.random.choice(list(solution))
    k = np.random.choice(list(solution))
    # Generating two different random positions
    while (i == k):
        k = np.random.choice(list(solution))
    # Switch between job i and job k in the given sequence
    neighbor = solution.copy()
    return swap(neighbor, i, k), evaluate_sequence(neighbor, processing_times)

In [35]:
def or_opt(solution, lenInterv, processing_times): 
    num_jobs = len(processing_times)
    
    # [i,j] les positions respectives des éléments à déplacer dans la séquence de la solution
    i = random.randint(0, num_jobs - lenInterv)
    j = i + lenInterv
        
    #sélectionner k, la position où insérer la séquence déplacée
    k = random.randint(0, num_jobs-1)

    neighbor = solution.copy()
    # Extraire la séquence entre i et j
    seq = neighbor[i:j+1]
    
    # Retirer la séquence de la solution
    neighbor = neighbor[:i] + neighbor[j+1:]
    
    # Insérer la séquence après k
    neighbor = neighbor[:k+1] + seq + neighbor[k+1:]
    
    return neighbor, evaluate_sequence(neighbor, processing_times)

In [36]:
def two_opt(solution, lenInterv, processing_times):
    
    num_jobs = len(processing_times)
    # [i,j] les positions respectives des éléments à déplacer dans la séquence de la solution
    i = random.randint(0, num_jobs - lenInterv)
    j = i + lenInterv
    
    #sélectionner k, la position où insérer la séquence déplacée
    k = random.randint(0, num_jobs-1)
    
    neighbor = solution.copy()
    # Extraire la séquence entre i et j et l'inverser
    seq = list(reversed(neighbor[i:j+1]))
    
    # Retirer la séquence de la solution
    neighbor = neighbor[:i] + neighbor[j+1:]

    # Insérer la séquence après k
    neighbor = neighbor[:k+1] + seq + neighbor[k+1:]
    
    return neighbor, evaluate_sequence(neighbor, processing_times)

### Perturbation function
This function performs a random perturbation on a list by exchanging the elements of k adjacent positions.

In [37]:
# solution: la liste à perturber
# k: le nombre de positions adjacentes à échanger

def shake(solution, k):
    n = len(solution)
    # Si k supérieur à la longueur de la liste, alors on crée des perturbations sur tous les éléments.
    indices = random.sample(range(n), min(k, n-1))
    indices.sort()
    neighbor = solution.copy()
    for i in indices:
        j = (i+k) % n
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

In [38]:
def intensification(population, processing_times):
    
    #On commence par sélectionner la sous-population sur laquelle on appliquera l'intensification    
    #La taille de la sous_pop est égale au nombre de jobs. 
    num_jobs = len(processing_times)
    
    #La moitié de la sous-population est représentée par les meilleurs solutions.
    population.sort(key=lambda x: x[1])
    sous_pop = population[:int(num_jobs/2)]
    
    #l'autre moitié sera choisie au hasard
    choix = population[int(num_jobs/2):]
    seq = random.sample(choix, int(num_jobs/2))
    sous_pop = sous_pop + seq

    
    #Maintenant nous allons appliquer un VNS sur chacune des populations résultantes
    for S in sous_pop:
        #Si la solution est considérée comme ancienne, nous la remplaçons par une solution gérérée par GRNEH sans itérations
        #Sauf pour la meilleure solution courrante
        if (S[2]>=8): 
            if (S != min(population, key=lambda x: x[1])):
                sol, cmax = GRNEH(processing_times, 5)
                B = (sol, cmax, 0)
                while (sol_unique(population, B[0])==False): 
                    B = (shake(B[0], 1), evaluate_sequence(B[0], processing_times), 0)
                replace_sol ( population, S, B)
        
        else:
            #B va stocker l'optimum local trouvé par VNS
            B = S 
                
            if ( 4 <= S[2] < 8 ):
                B = ( shake(B[0], 2), evaluate_sequence(B[0], processing_times), B[2])
        
            for loop in range ( (num_jobs)*(num_jobs-1) ):
                if (S[2]== 0 or S[2]== 4): 
                    sol, cmax = random_swap(B[0], processing_times)
                    ST = (sol, cmax, 0)
                if (S[2]== 1 or S[2]== 5):
                    sol, cmax = random_insertion(B[0], processing_times)
                    ST = (sol, cmax, 0)
                if (S[2]== 2 or S[2]== 6): 
                    sol, cmax = or_opt(B[0], 3, processing_times)
                    ST = (sol, cmax, 0)
                if (S[2]== 3 or S[2]== 7):
                    sol, cmax = two_opt(B[0], 3, processing_times)
                    ST = (sol, cmax, 0)
                if ( ST[1] <= B[1] ): 
                    B = ST
         
        #Si l'optimum local retournée par VNS améliore la solution i, alors on vérifie qu'il est unique puis on remplace
        #OBSERVATION : Si on met <=, on aura toujours des remplacement et bcp de solution semblable
            if ( B[1] < S[1] ): 
                if (sol_unique(population, B[0])): 
                    replace_sol ( population, S, B)

                else:
                    replace_sol ( population, S, (S[0], S[1], S[2]+1))        

            else: 
                replace_sol ( population, S, (S[0], S[1], S[2]+1))        
    
    best_indiv = min(population, key=lambda x: x[1])
    
    return best_indiv[0], best_indiv[1]

In [39]:
def NEGA(population, processing_times, num_iterations):
    for i in range(num_iterations):
        AG(population, processing_times)
        best_sol, best_fitness = intensification(population, processing_times)
    return best_sol, best_fitness

# Tests on Taillard instances

## Reading Taillard instances

In [63]:
# Open the file that contains the instances
file = open("Benchmarks/tai20_5.txt", "r")

# Read the file line by line to retrieve the instances
n = 0
instances_20_5 = [[]]
line = file.readline()

while line:
    if line != '\n':
        line = line.strip(' ')
        line = line[:-1]
        line = line.split()
        line = [int(num) for num in line]
        instances_20_5[n].append(line)
    else:
        instances_20_5.append([])
        n += 1
    line = file.readline()
    
print(f'Taillard, 20 jobs 5 machines contains {len(instances_20_5)} benchmark.')  

Taillard, 20 jobs 5 machines contains 10 benchmark.


In [64]:
# Open the file that contains the instances
file = open("Benchmarks/tai50_10.txt", "r")

# Read the file line by line to retrieve the instances
n = 0
instances_50_10 = [[]]
line = file.readline()

while line:
    if line != '\n':
        line = line.strip(' ')
        line = line[:-1]
        line = line.split()
        line = [int(num) for num in line]
        instances_50_10[n].append(line)
    else:
        instances_50_10.append([])
        n += 1
    line = file.readline()
    
print(f'Taillard, 50 jobs 10 machines contains {len(instances_50_10)} benchmark.')

Taillard, 50 jobs 10 machines contains 10 benchmark.


In [65]:
# Open the file that contains the instances
file = open("Benchmarks/tai100_10.txt", "r")

# Read the file line by line to retrieve the instances
n = 0
instances_100_10 = [[]]
line = file.readline()

while line:
    if line != '\n':
        line = line.strip(' ')
        line = line[:-1]
        line = line.split()
        line = [int(num) for num in line]
        instances_100_10[n].append(line)
    else:
        instances_100_10.append([])
        n += 1
    line = file.readline()
    
print(f'Taillard, 100 jobs 10 machines contains {len(instances_100_10)} benchmark.')

Taillard, 100 jobs 10 machines contains 10 benchmark.


In [66]:
# Open the file that contains the instances
file = open("Benchmarks/tai200_10.txt", "r")

# Read the file line by line to retrieve the instances
n = 0
instances_200_10 = [[]]
line = file.readline()

while line:
    if line != '\n':
        line = line.strip(' ')
        line = line[:-1]
        line = line.split()
        line = [int(num) for num in line]
        instances_200_10[n].append(line)
    else:
        instances_200_10.append([])
        n += 1
    line = file.readline()
    
print(f'Taillard, 200 jobs 10 machines contains {len(instances_200_10)} benchmark.')

Taillard, 200 jobs 10 machines contains 10 benchmark.


In [67]:
instance_20_5_0 = np.array(instances_20_5[0])
instance_20_5_0 = instance_20_5_0.T

In [45]:
instance_50_10_0 = np.array(instances_50_10[0])
instance_50_10_0 = instance_50_10_0.T

In [46]:
instance_100_10_0 = np.array(instances_100_10[0])
instance_100_10_0 = instance_100_10_0.T

## Hybrid GA and VNS Algorithm

### First instance of the 20_5 benchmark

In [68]:
popul = init_pop(instance_20_5_0, 50, 0.5)
start_time = time.time()
Best_solution, Best_makespan = NEGA(popul, instance_20_5_0, 50)
elapsed_time = time.time() - start_time

print(f'Solution: {Best_solution}.')
print(f'\nCmax: {Best_makespan}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [8, 14, 16, 12, 7, 18, 5, 3, 10, 4, 2, 6, 0, 1, 17, 13, 15, 9, 19, 11].

Cmax: 1278.0

Elapsed time:  197.775652885437 seconds


Here we fix the number of iterations to 200. The best one found above

In [74]:
popul = init_pop(instance_20_5_0, 50, 0.5)
start_time = time.time()
Best_solution, Best_makespan = NEGA(popul, instance_20_5_0, 200)
elapsed_time = time.time() - start_time

print(f'Solution: {Best_solution}.')
print(f'\nCmax: {Best_makespan}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [8, 14, 5, 2, 16, 13, 18, 12, 7, 10, 4, 0, 6, 15, 17, 3, 1, 9, 19, 11].

Cmax: 1278.0

Elapsed time:  281.09008026123047 seconds


In [78]:
popul = init_pop(instance_20_5_0, 100, 0.5)
start_time = time.time()
Best_solution, Best_makespan = NEGA(popul, instance_20_5_0, 200)
elapsed_time = time.time() - start_time

print(f'Solution: {Best_solution}.')
print(f'\nCmax: {Best_makespan}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [2, 7, 16, 8, 5, 18, 14, 13, 10, 12, 6, 4, 3, 1, 17, 15, 0, 9, 19, 11].

Cmax: 1278.0

Elapsed time:  305.90509963035583 seconds


In [80]:
popul = init_pop(instance_20_5_0, 150, 0.5)
start_time = time.time()
Best_solution, Best_makespan = NEGA(popul, instance_20_5_0, 200)
elapsed_time = time.time() - start_time

print(f'Solution: {Best_solution}.')
print(f'\nCmax: {Best_makespan}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [8, 14, 5, 18, 13, 7, 4, 3, 16, 17, 15, 0, 1, 2, 6, 10, 12, 9, 19, 11].

Cmax: 1278.0

Elapsed time:  745.8400273323059 seconds


In [89]:
popul = init_pop(instance_20_5_0, 50, 0.5)
start_time = time.time()
Best_solution, Best_makespan = NEGA(popul, instance_20_5_0, 200)
elapsed_time = time.time() - start_time

print(f'Solution: {Best_solution}.')
print(f'\nCmax: {Best_makespan}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [8, 16, 2, 13, 10, 14, 5, 17, 3, 18, 7, 4, 12, 15, 6, 0, 1, 9, 19, 11].

Cmax: 1278.0

Elapsed time: 725.8700274323051 seconds


In [90]:
popul = init_pop(instance_20_5_0, 50, 0.3)
start_time = time.time()
Best_solution, Best_makespan = NEGA(popul, instance_20_5_0, 200)
elapsed_time = time.time() - start_time

print(f'Solution: {Best_solution}.')
print(f'\nCmax: {Best_makespan}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [8, 14, 5, 16, 3, 10, 12, 13, 18, 4, 0, 2, 1, 17, 6, 7, 15, 9, 19, 11].

Cmax: 1278.0

Elapsed time: 737.14700278923021 seconds


### First instance of the 50_10 benchmark

In [51]:
popul = init_pop(instance_50_10_0, 50, 0.12)
start_time = time.time()
Best_solution, Best_makespan = NEGA(popul, instance_50_10_0, 2)
elapsed_time = time.time() - start_time

print(f'Solution: {Best_solution}.')
print(f'\nCmax: {Best_makespan}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [17, 21, 41, 43, 42, 35, 30, 1, 33, 32, 48, 14, 40, 19, 28, 46, 3, 13, 39, 37, 25, 24, 36, 26, 11, 22, 16, 6, 8, 2, 4, 31, 5, 10, 12, 29, 49, 45, 27, 9, 15, 20, 18, 7, 34, 0, 23, 47, 44, 38].

Cmax: 3063.0

Elapsed time:  566.2167835235596 seconds


## Genetic algorithm

### First instance of the 20_5 benchmark

In [52]:
pop_size = 30
select_pop_size = 0.5
selection_method = "roulette"
cossover = "TWO"
mutation_probability = .5
num_iterations = 100

# Checking parameters
s = int(select_pop_size * pop_size)
try:
    if s < 2:
        raise ValueError("there should be at least two selected parents : please increase the % of selection")
    comb = math.comb(s, 2) * 2
    if comb < pop_size:
        raise ValueError("The maximum size of population with the choosed parameters is "+str(comb)+" sequences : please increase the % of selection or decrease the size of population")   
    init_pop = [np.random.permutation(instance_20_5_0.shape[0]).tolist() for i in range(pop_size)]
    start_time = time.time()
    best_solution, best_solution_cost = genetic_algorithm(instance_20_5_0, init_pop, pop_size, select_pop_size, selection_method, cossover, mutation_probability, num_iterations)
    elapsed_time = time.time() - start_time
    print(f'Best sequence found by genetic algorithm is {best_solution} with a makespan of {best_solution_cost}')
    print("Elapsed time:", elapsed_time, "seconds")      
except ValueError as e:
    print(e)

Best sequence found by genetic algorithm is [4, 8, 12, 7, 16, 15, 3, 14, 5, 0, 1, 11, 18, 13, 17, 6, 19, 10, 2, 9] with a makespan of 1343.0
Elapsed time: 2.6246378421783447 seconds


### First instance of the 50_10 benchmark

In [53]:
pop_size = 30
select_pop_size = 0.5
selection_method = "roulette"
cossover = "TWO"
mutation_probability = .5
num_iterations = 100

# Checking parameters
s = int(select_pop_size * pop_size)
try:
    if s < 2:
        raise ValueError("there should be at least two selected parents : please increase the % of selection")
    comb = math.comb(s, 2) * 2
    if comb < pop_size:
        raise ValueError("The maximum size of population with the choosed parameters is "+str(comb)+" sequences : please increase the % of selection or decrease the size of population")   
    init_pop = [np.random.permutation(instance_50_10_0.shape[0]).tolist() for i in range(pop_size)]
    start_time = time.time()
    best_solution, best_solution_cost = genetic_algorithm(instance_50_10_0, init_pop, pop_size, select_pop_size, selection_method, cossover, mutation_probability, num_iterations)
    elapsed_time = time.time() - start_time
    print(f'Best sequence found by genetic algorithm is {best_solution} with a makespan of {best_solution_cost}')
    print("Elapsed time:", elapsed_time, "seconds")      
except ValueError as e:
    print(e)

Best sequence found by genetic algorithm is [21, 48, 0, 14, 27, 32, 28, 18, 42, 15, 45, 29, 40, 11, 39, 9, 41, 17, 47, 3, 19, 7, 13, 30, 43, 35, 49, 34, 12, 5, 16, 8, 22, 24, 31, 23, 46, 20, 44, 33, 37, 4, 38, 25, 10, 26, 1, 36, 6, 2] with a makespan of 3477.0
Elapsed time: 13.402140378952026 seconds


## Ant colony

### First instance of the 20_5 benchmark

In [81]:
num_ants = 10
num_iterations = 50
tau0 = 1.0
alpha = 1.0
beta = 2.0
Q = 1.0
evaporation_rate = .9
q0 = 0.4
start_time = time.time()
best_solution, best_solution_cost = ant_colony_optimization(num_ants, num_iterations, alpha, beta, evaporation_rate, Q, tau0, q0, instance_20_5_0)
elapsed_time = time.time() - start_time
print(f'Best sequence found by ACO algorithm is {best_solution} with a makespan of {best_solution_cost}')
print("Elapsed time:", elapsed_time, "seconds")

Best sequence found by ACO algorithm is [13, 3, 0, 18, 17, 5, 16, 6, 14, 1, 8, 11, 7, 10, 19, 4, 2, 12, 15, 9] with a makespan of 1394.0
Elapsed time: 1.40116286277771 seconds


### First instance of the 50_10 benchmark

In [70]:
num_ants = 10
num_iterations = 50
tau0 = 1.0
alpha = 1.0
beta = 2.0
Q = 1.0
evaporation_rate = .9
q0 = 0.6

start_time = time.time()
best_solution, best_solution_cost = ant_colony_optimization(num_ants, num_iterations, alpha, beta, evaporation_rate, Q, tau0, q0, instance_50_10_0)
elapsed_time = time.time() - start_time
print(f'Best sequence found by ACO algorithm is {best_solution} with a makespan of {best_solution_cost}')
print("Elapsed time:", elapsed_time, "seconds")

Best sequence found by ACO algorithm is [41, 43, 36, 24, 26, 22, 21, 1, 13, 39, 30, 9, 33, 44, 35, 31, 27, 48, 23, 45, 38, 4, 49, 7, 14, 34, 25, 10, 2, 20, 28, 37, 6, 5, 8, 46, 15, 16, 18, 40, 0, 42, 47, 3, 32, 19, 29, 17, 12, 11] with a makespan of 3564.0
Elapsed time: 7.4078309535980225 seconds


## Heuristics and local search heuristics

### NEH

#### First instance of the 20_5 benchmark

In [71]:
start_time = time.time()
neh_20_5_0, Cmax = neh_algorithm(instance_20_5_0)
elapsed_time = time.time() - start_time

print(f'Solution: {neh_20_5_0}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19].

Cmax: 1334.0

Elapsed time:  0.04700875282287598 seconds


#### First instance of the 50_10 benchmark

In [73]:
start_time = time.time()
neh_50_10_0, Cmax = neh_algorithm(instance_50_10_0)
elapsed_time = time.time() - start_time

print(f'Solution: {neh_50_10_0}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [19, 33, 30, 14, 35, 13, 41, 34, 9, 43, 32, 42, 24, 11, 39, 46, 17, 47, 12, 40, 37, 8, 4, 45, 28, 1, 2, 10, 44, 21, 5, 3, 48, 16, 31, 29, 25, 49, 23, 0, 15, 36, 7, 27, 6, 22, 20, 38, 26, 18].

Cmax: 3229.0

Elapsed time:  1.7784607410430908 seconds


### Artificial Heuristic

#### First instance of the 20_5 benchmark

In [74]:
start_time = time.time()
sol, Cmax = artificial_heuristic(instance_20_5_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [2, 8, 14, 16, 13, 7, 18, 5, 10, 15, 1, 3, 4, 0, 11, 6, 17, 9, 19, 12].

Cmax: 1367.0

Elapsed time:  0.0009951591491699219 seconds


#### First instance of the 50_10 benchmark

In [75]:
start_time = time.time()
sol, Cmax = artificial_heuristic(instance_50_10_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Solution: [41, 17, 43, 32, 19, 29, 36, 35, 48, 21, 24, 13, 33, 30, 28, 42, 37, 1, 3, 14, 2, 15, 45, 27, 8, 39, 22, 10, 16, 4, 12, 46, 9, 5, 11, 40, 31, 44, 20, 7, 25, 34, 6, 49, 18, 0, 47, 23, 38, 26].

Cmax: 3461.0

Elapsed time:  0.014571428298950195 seconds


### VNS

#### First instance of the 20_5 benchmark

In [76]:
start_time = time.time()
sol, Cmax = vns(neh_20_5_0, instance_20_5_0, 100, 5)
elapsed_time = time.time() - start_time

print(f'\nGenerated Solution: {sol}.')
print(f'Makespan: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")


Generated Solution: [8, 12, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 4, 11, 0, 18, 5, 9, 6, 19].
Makespan: 1305.0

Elapsed time:  0.27494072914123535 seconds


#### First instance of the 50_10 benchmark

In [77]:
start_time = time.time()
sol, Cmax = vns(neh_50_10_0, instance_50_10_0, 100, 5)
elapsed_time = time.time() - start_time

print(f'\nGenerated Solution: {sol}.')
print(f'Makespan: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")


Generated Solution: [19, 33, 30, 14, 35, 13, 41, 34, 9, 43, 32, 42, 24, 28, 39, 46, 17, 47, 12, 40, 37, 3, 4, 2, 22, 1, 45, 10, 44, 29, 5, 8, 6, 31, 16, 21, 23, 49, 25, 0, 15, 36, 7, 27, 48, 11, 20, 26, 18, 38].
Makespan: 3196.0

Elapsed time:  1.3540894985198975 seconds


## Results

In [82]:
df = pd.DataFrame({'Algorithm': pd.Series(dtype='str'),
                   '20-5-1 (Makespan)': pd.Series(dtype='str'),
                   '20-5-1 (Time)': pd.Series(dtype='str'),
                   '50-10-1 (Makespan)': pd.Series(dtype='str'),
                   '50-10-1 (Time)': pd.Series(dtype='str'),
                  }) 

In [83]:
GA = ['Genetic Algorithm', '1334', '2.627s', '3477', '13.40s']
ACO = ['Ant colony', '1394', '1.40s', '3564', '7.40s']
Hybrid = ['Hybrid', '1278', '197.77s', '3063', '566.21s']
NEH = ['NEH', '1334', '0.04s', '3229', '1.77s']
AH = ['Artificial heuristic', '1367', '0.0s', '3461', '0.01s']
VNS = ['Ant colony', '1305', '0.27s', '3196', '1.35s']

In [84]:
df.loc[0]= GA
df.loc[1]= ACO
df.loc[2]= Hybrid
df.loc[3]= NEH
df.loc[4]= AH
df.loc[5]= VNS

In [85]:
df

,Algorithm,20-5-1 (Makespan),20-5-1 (Time),50-10-1 (Makespan),50-10-1 (Time)
0,Genetic Algorithm,1334,2.627s,3477,13.40s
1,Ant colony,1394,1.40s,3564,7.40s
2,Hybrid,1278,197.77s,3063,566.21s
3,NEH,1334,0.04s,3229,1.77s
4,Artificial heuristic,1367,0.0s,3461,0.01s
5,Ant colony,1305,0.27s,3196,1.35s
